In [15]:
import os
import numpy as np
import pandas as pd
import time
import torch
from Utils.DataUtil import readData, readJson
import math
from datetime import  datetime
import os
import random
import argparse
from Utils.EvalUtil import findSegment
from Utils.PlotUtil import plotAllResult
from importlib import import_module

In [16]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 使用一个固定的种子
set_seed(42)

In [17]:

def parseParams():
    parser = argparse.ArgumentParser(description='Time series anomaly detection system')

    parser.add_argument('--random_seed', type=int, default=42, help='random seed')
    parser.add_argument('--model_name', type=str, default="MSCRED", help='name of model')
    parser.add_argument('--dataset', type=str, default="NASA", help="name of dataset,like 'NASA'")
    parser.add_argument('--filename', type=str, default="M-1", help="file-name of time series ")
    parser.add_argument('--filetype', type=str, default="npy", help="file-type of time series")

    parser.add_argument('--channels', type=int, default=55, help="nums of dimension for time series")

    parser.add_argument('--epoch', type=int, default=20, help="num of training epoches")
    parser.add_argument('--learning_rate', type=float, default=0.001, help="value of learning rate")
    parser.add_argument('--batch_size', type=int, default=128, help="batch size of data")
    parser.add_argument('--shuffle', type=bool, default=False, help="whether do shuffle by time window")

    args = parser.parse_args(args=[])

    return args

def getConfig(args):

    start_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

    identifier = args.model_name + "/" + start_time
    config = {
        "base_path":"../",
        "model": args.model_name,
        "dataset":args.dataset,
        "filename":args.filename,
        "filetype":args.filetype,
        "epoch": args.epoch,
        "input_size": args.channels,
        "learning_rate": args.learning_rate,
        "identifier": identifier,
        "batch_size": args.batch_size,
        "device" :torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "shuffle":args.shuffle
    }

    model_config = readJson(path = config["base_path"] + "/Models/"+config["model"]+"/Config.json")

    config = { **model_config[config["dataset"]][config["filename"]],** config }

    #fix random seed
    # fix_seed = args.random_seed
    # torch.manual_seed(fix_seed)
    # np.random.seed(fix_seed)




    return config


def getModel(config):
    method = config["model"]
    module = import_module("Models."+method+".Model")
    # 获取类引用
    clazz = getattr(module, method)

    # 创建类的实例
    model = clazz(config).float()
    # model = model_dict[method].Model(args).float()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    return model

In [18]:
# data_label_1 = pd.read_csv("../Data/SMD/label/machine-1-1.txt",header=None)
# data_label_1.plot()

In [19]:
# data_test_1 = pd.read_csv("../Data/SMD/test/machine-1-1.txt",header=None)
# selected_columns = [1,9,10]
# data_test_1 = data_test_1[selected_columns]
# data_test_1.plot()

采用全部38个维度的数据训练的模型 5个epoch  F1: f1-score: 0.09188588366061504 100阈值:f1-score: 0.4449230769230769

In [20]:
args = parseParams()
args.model_name = "TRANSFORMER"
config = getConfig(args=args)
config["hidden_size"] = 80
config["num_layers"] = 2
config["num_heads"] = 1
config["drop_out_rate"] = 0.2
config["latent_size"] = 14
config["epoch"] = 5


window_size = config["window_size"]



#preprocess data
label = pd.read_csv("../Data/SMD/label/machine-1-1.txt",header=None).to_numpy()[window_size - 1:]
data_test = pd.read_csv("../Data/SMD/test/machine-1-1.txt",header=None).to_numpy()
data_train = pd.read_csv("../Data/SMD/train/machine-1-1.txt",header=None).to_numpy()


config["input_size"] = data_test.shape[-1]
#get data
print(config)

print("data_train shape:",data_train.shape)
print("data_test shape:", data_test.shape)

device = config["device"]

#get model
model = getModel(config=config).to(device)
print("model:",model)

shuffle = config["shuffle"]


#train model
model.fit(train_data= data_train,write_log=True)

#get anomaly score
# model.setThreshold(data_train,data_test,label)
anomaly_scores1 = model.test(data_test)
print("anomaly score:",anomaly_scores1)
#predict anomaly based on the threshold
# threshold = model.getThreshold()
#
# predict_labels =  model.decide(anomaly_score=anomaly_scores,threshold=threshold,ground_truth_label=label)
# # result = model.predictEvaluate(test_data=data_test, label = label, protocol ="apa" )
# # print(result)
#
# # #evaluate
# f1 = model.evaluate(predict_label=predict_labels,ground_truth_label=label,threshold=threshold,write_log=False)


predict_labels,f1,threshold = model.getBestPredict(anomaly_score=anomaly_scores1,n_thresholds = 100,ground_truth_label=label,protocol="none")
print("f1-score:", f1," threshold:",threshold)

#
# #visualization
# plot_yaxis = []
# plot_yaxis.append(anomaly_scores1)
# plot_yaxis.append(predict_labels)
# plot_path = config["base_path"]+"/Plots/"+config["identifier"]
# # 判断文件夹是否存在
# if not os.path.exists(plot_path):
#     # 如果文件夹不存在，则创建它
#     os.makedirs(plot_path)
# plotAllResult(x_axis=np.arange(len(predict_labels)), y_axises=plot_yaxis, title="",
#                 save_path=plot_path+"/result.pdf", segments=findSegment(label),threshold=threshold)


{'hidden_size': 80, 'latent_size': 14, 'window_size': 60, 'threshold': 0.5, 'batch_size': 128, 'num_layers': 2, 'num_heads': 1, 'drop_out_rate': 0.2, 'input_size': 38, 'epoch': 5, 'base_path': '../', 'model': 'TRANSFORMER', 'dataset': 'NASA', 'filename': 'M-1', 'filetype': 'npy', 'learning_rate': 0.001, 'identifier': 'TRANSFORMER/2024-07-06-10-52-07', 'device': device(type='cpu'), 'shuffle': False}
data_train shape: (28479, 38)
data_test shape: (28479, 38)
model: TRANSFORMER(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=38, out_features=38, bias=True)
          )
          (linear1): Linear(in_features=38, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=256, out_features=38, bias=True)
          (norm1): LayerNorm((38,), 


采用后面27个维度的数据训练的模型 5个epoch 25阈值 f1-score: 0.09020332717190388 100阈值： f1-score: 0.24173491052471943 threshold: 0.010101010101010102



In [21]:
args = parseParams()
args.model_name = "TRANSFORMER"
config = getConfig(args=args)
config["hidden_size"] = 80
config["num_layers"] = 2
config["num_heads"] = 1
config["drop_out_rate"] = 0.2
config["latent_size"] = 14
config["epoch"] = 5


window_size = config["window_size"]



#preprocess data
label = pd.read_csv("../Data/SMD/label/machine-1-1.txt",header=None).to_numpy()[window_size - 1:]
data_test = pd.read_csv("../Data/SMD/test/machine-1-1.txt",header=None).to_numpy()[:,11:]
data_train = pd.read_csv("../Data/SMD/train/machine-1-1.txt",header=None).to_numpy()[:,11:]


config["input_size"] = data_test.shape[-1]
#get data
print(config)

print("data_train shape:",data_train.shape)
print("data_test shape:", data_test.shape)

device = config["device"]

#get model
model = getModel(config=config).to(device)
print("model:",model)

shuffle = config["shuffle"]


#train model
model.fit(train_data= data_train,write_log=True)

#get anomaly score
# model.setThreshold(data_train,data_test,label)
anomaly_scores2 = model.test(data_test)
print("anomaly score:",anomaly_scores2)
#predict anomaly based on the threshold
# threshold = 0.041666666666666664
#
# predict_labels =  model.decide(anomaly_score=anomaly_scores,threshold=threshold,ground_truth_label=label)
# # result = model.predictEvaluate(test_data=data_test, label = label, protocol ="apa" )
# # print(result)
#
# # #evaluate
# f1 = model.evaluate(predict_label=predict_labels,ground_truth_label=label,threshold=threshold,write_log=False)


predict_labels,f1,threshold = model.getBestPredict(anomaly_score=anomaly_scores2,n_thresholds = 100,ground_truth_label=label,protocol="none")
model.save()
print("f1-score:", f1,"threshold:",threshold)


# #visualization
# plot_yaxis = []
# plot_yaxis.append(anomaly_scores2)
# plot_yaxis.append(predict_labels)
# plot_path = config["base_path"]+"/Plots/"+config["identifier"]
# # 判断文件夹是否存在
# if not os.path.exists(plot_path):
#     # 如果文件夹不存在，则创建它
#     os.makedirs(plot_path)
# plotAllResult(x_axis=np.arange(len(predict_labels)), y_axises=plot_yaxis, title="",
#                 save_path=plot_path+"/result.pdf", segments=findSegment(label),threshold=threshold)



{'hidden_size': 80, 'latent_size': 14, 'window_size': 60, 'threshold': 0.5, 'batch_size': 128, 'num_layers': 2, 'num_heads': 1, 'drop_out_rate': 0.2, 'input_size': 27, 'epoch': 5, 'base_path': '../', 'model': 'TRANSFORMER', 'dataset': 'NASA', 'filename': 'M-1', 'filetype': 'npy', 'learning_rate': 0.001, 'identifier': 'TRANSFORMER/2024-07-06-10-54-51', 'device': device(type='cpu'), 'shuffle': False}
data_train shape: (28479, 27)
data_test shape: (28479, 27)
model: TRANSFORMER(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=27, out_features=27, bias=True)
          )
          (linear1): Linear(in_features=27, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=256, out_features=27, bias=True)
          (norm1): LayerNorm((27,), 

In [22]:
from torch.optim.lr_scheduler import CosineAnnealingLR

model = getModel(config=config).to(device)
model.load_state_dict(torch.load("../CheckPoints/TRANSFORMER/2024-07-04-15-41-55/checkpoint.pth"))
print(model)

model.input_adpter = torch.nn.Linear(38,27)
model.output_adpter = torch.nn.Linear(27,38)

window_size = config["window_size"]
#preprocess data
label = pd.read_csv("../Data/SMD/label/machine-1-1.txt",header=None).to_numpy()[window_size - 1:]
data_test = pd.read_csv("../Data/SMD/test/machine-1-1.txt",header=None).to_numpy()
data_train = pd.read_csv("../Data/SMD/train/machine-1-1.txt",header=None).to_numpy()




train_loader = model.processData(data_train)
model.train()
lr = model.config["learning_rate"]
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

# 设置余弦学习率衰减，这里的T_max是衰减周期
scheduler = CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-7)


l = torch.nn.MSELoss(reduction='sum')

epoch_loss = []


for ep in range(5):

    # l1s = []
    running_loss = 0
    for d in train_loader:
        optimizer.zero_grad()
        item = d[0].to(model.divice)


        data_adpted = model.input_adpter(item)
        output = model(data_adpted,data_adpted[:,-1,:].unsqueeze(dim=1))
        output = model.output_adpter(output)

        loss = l(output, item[:,-1,:].unsqueeze(dim=1))



        # l1s.append(torch.mean(loss).item())

        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    scheduler.step()  # 在每个epoch后更新学习率

    # 计算当前epoch的平均损失
    epoch_loss.append(running_loss / len(train_loader))

    print(f'train epoch [{ep+1}/{model.epoch}],\t loss = {epoch_loss[ep]}')


TRANSFORMER(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=27, out_features=27, bias=True)
          )
          (linear1): Linear(in_features=27, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=256, out_features=27, bias=True)
          (norm1): LayerNorm((27,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((27,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((27,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0): TransformerDecoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj

In [23]:
from Preprocess.Normalization import minMaxScaling

test_dataloader = model.processData(data_test)
model.eval()
score = []

l = torch.nn.MSELoss(reduction='none')

with torch.no_grad():
    for index, d in enumerate(test_dataloader):
        item = d[0].to(model.divice)

        data_adpted = model.input_adpter(item)
        output = model(data_adpted,data_adpted[:,-1,:].unsqueeze(dim=1))

        output = model.output_adpter(output)

        loss = l(output[:, -1, :], item[:, -1, :])

        loss = loss.sum(dim=-1)
        if len(loss.shape) == 0:
            loss = loss.unsqueeze(dim=0)

        score.append(loss.detach().cpu())


    score = torch.concatenate(score, dim=0).numpy()

    score = minMaxScaling(data=score, min_value=score.min(), max_value=score.max(), range_max=1, range_min=0)

print(score)


[0.00027793 0.00014188 0.00012081 ... 0.00092707 0.00104166 0.00386006]


In [24]:
# threshold = model.getThreshold()
#
# predict_labels =  model.decide(anomaly_score=score,threshold=threshold,ground_truth_label=label,protocol="none")
#
# print(findSegment(predict_labels))
# # print(result)
#
# # #evaluate
# f1 = model.evaluate(predict_label=predict_labels,ground_truth_label=label,threshold=threshold,write_log=False)
# #
#
predict_labels,f1,threshold = model.getBestPredict(anomaly_score=score,n_thresholds = 100,ground_truth_label=label,protocol="none")
print("f1-score:", f1)
print("threshold:",threshold)
#visualization
# plot_yaxis = []
# plot_yaxis.append(score)
# plot_yaxis.append(predict_labels)
# plot_path = config["base_path"]+"/Plots/"+config["identifier"]
# # 判断文件夹是否存在
# if not os.path.exists(plot_path):
#     # 如果文件夹不存在，则创建它
#     os.makedirs(plot_path)
# plotAllResult(x_axis=np.arange(len(predict_labels)), y_axises=plot_yaxis, title="",
#                 save_path=plot_path+"/result.pdf", segments=findSegment(label),
#                 threshold=threshold)


f1-score: 0.4520921770770164
threshold: 0.010101010101010102
